In [107]:
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel
from coco_dataset import COCODataset, transform_image
from copy import deepcopy
import random
import pandas as pd
from tqdm import tqdm
import sys
import os
from train_utils import chunk_data

/tmp/ipykernel_8062/1793954293.py:5: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


Define number of keypoints of new neural network

In [108]:
NO_OF_KEYPOINTS = 17

In [109]:
path_to_coco_dataset = '/media/jakub/One Touch/coco_pose/coco2017labels-pose/coco-pose'

In [110]:
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
            [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
            [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85], [255, 255, 255]]

Create backbone model

In [111]:
backbone_model_path = '../model/body_pose_model.pth'

In [112]:
backbone_model = BodyposeBackbone()

Load backbone params from file

In [113]:
backbone_params = torch.load(backbone_model_path)

In [114]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [115]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [116]:
main_model = MainModel(backbone_model, NO_OF_KEYPOINTS)

Create dataset from COCO .json file

In [117]:
train_data = COCODataset(path_to_coco_dataset, transforms=transform_image(train=True))

loading annotations into memory...
Done (t=0.43s)
creating index...
index created!


In [118]:
# Skeleton defines which joints are connected. Because of numbering from 1 I subtract it
skeleton = train_data.coco.cats[1]['skeleton']
skeleton = [[connection[0]-1, connection[1]-1] for connection in skeleton]

In [119]:
# create list with ids where human is marked
valid_ids = []

In [120]:

for id in train_data.coco.getAnnIds():
    ann = train_data.coco.anns[id]
    if 1 in ann['keypoints'] or 2 in ann['keypoints']:
        valid_ids.append(ann['image_id'])


Show example image

In [121]:
index = random.randint(0, 100)
# load image
torch_image, image, target= train_data[index]

reference_image = image.copy()
model_output_image = image.copy()
# draw keypoints
for keypoint in target:
    for point in keypoint:
        x = point[0]
        y = point[1]
        if x != 0 and y != 0:
            cv2.circle(reference_image, (x, y), 3, (0, 0, 255), -1)
cv2.imshow('Example image', reference_image)
cv2.waitKey(10_000)
cv2.destroyAllWindows()

In [122]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [123]:
device = torch.device("cpu")

In [124]:
# image as torch tensor with shape as (CxHxW)
torch_image.size()

torch.Size([3, 480, 640])

In [125]:
output = main_model(torch_image)
part_affinity_field_maps, confidence = output
part_affinity_field_maps = part_affinity_field_maps.detach().numpy()

In [126]:
part_affinity_field_maps = np.average(part_affinity_field_maps, axis=1)
part_affinity_field_maps = np.average(part_affinity_field_maps, axis=1)

In [127]:
# chunk the data
part_affinity_field_maps_chunk = chunk_data(part_affinity_field_maps, 2)

for point in part_affinity_field_maps_chunk:
    x = point[0]
    y = point[1]
    if x != 0 and y != 0:
        cv2.circle(model_output_image, (int(x), int(y)), 3, (255, 0, 0), -1)
cv2.imshow('Example image', model_output_image)
cv2.waitKey(10_000)
cv2.destroyAllWindows()

In [128]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [129]:
def train(main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()

    main_model.train()

    logits, confidence = main_model(inputs)
    # logits = logits.detach().numpy()
    logits = torch.mean(logits, axis=1)
    logits = torch.mean(logits, axis=1)
    logits = torch.FloatTensor(logits)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    # return logits
    return output.item()

In [130]:
epochs = 1

losses = []

for epoch in range(epochs):
    for id in valid_ids:
        print(f"epoch: {epoch}, id: {id}")
        torch_image, image, targets = train_data[id]
        targets = torch.FloatTensor(targets)
        if targets.size()[0] > 1:
            for target in targets:
                target = target[:,:-1].flatten()
                output = train(main_model=main_model, loss=loss, optimizer=optimizer, inputs=torch_image, outputs=target)
                print(f"epoch: {epoch}, id: {id}, output: {output}")
        else:
            targets = targets[:,:,:-1].flatten()
            output = train(main_model=main_model, loss=loss, optimizer=optimizer, inputs=torch_image, outputs=targets)
        print(output)

epoch: 0, id: 425226


IndexError: list index out of range

In [ ]:
# train_data.coco.imgs[425226]
for i, keypoint in enumerate(train_data.coco.loadAnns(train_data.coco.getAnnIds(425226))):
    print(keypoint)

{'segmentation': [[125.12, 539.69, 140.94, 522.43, 100.67, 496.54, 84.85, 469.21, 73.35, 450.52, 104.99, 342.65, 168.27, 290.88, 179.78, 288, 189.84, 286.56, 191.28, 260.67, 202.79, 240.54, 221.48, 237.66, 248.81, 243.42, 257.44, 256.36, 253.12, 262.11, 253.12, 275.06, 299.15, 233.35, 329.35, 207.46, 355.24, 206.02, 363.87, 206.02, 365.3, 210.34, 373.93, 221.84, 363.87, 226.16, 363.87, 237.66, 350.92, 237.66, 332.22, 234.79, 314.97, 249.17, 271.82, 313.89, 253.12, 326.83, 227.24, 352.72, 214.29, 357.03, 212.85, 372.85, 208.54, 395.87, 228.67, 414.56, 245.93, 421.75, 266.07, 424.63, 276.13, 437.57, 266.07, 450.52, 284.76, 464.9, 286.2, 479.28, 291.96, 489.35, 310.65, 512.36, 284.76, 549.75, 244.49, 522.43, 215.73, 546.88, 199.91, 558.38, 204.22, 565.57, 189.84, 568.45, 184.09, 575.64, 172.58, 578.52, 145.26, 567.01, 117.93, 551.19, 133.75, 532.49]], 'num_keypoints': 10, 'area': 47803.27955, 'iscrowd': 0, 'keypoints': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 142, 309, 1, 177, 320, 2

In [ ]:
torch_target = torch.FloatTensor(target)

In [ ]:
for target in torch_target:
    print(target.size())

torch.Size([17, 3])


In [ ]:
torch_target.size()

torch.Size([1, 17, 3])